# Configuração de ambiente

In [1]:
!mkdir Image
!mkdir Image_Segmentation

In [ ]:
%pip install crewai crewai-tools poetry vllm
!curl -fsSL https://ollama.com/install.sh | sh
!pip install pymupdf langchain-community langchain-core langgraph faiss-cpu
!pip install langchain --upgrade

In [3]:
!nohup ollama serve > ollama.log &
!nohup ollama run llava:7b &

nohup: redirecting stderr to stdout
nohup: appending output to 'nohup.out'


# Criação da RAG leve com base no PDF

In [ ]:
import pymupdf  # PyMuPDF
import os
import zipfile

# Caminho do arquivo ZIP enviado pelo usuário
zip_path = "RAG.zip"
extracted_path = "extracted_pdfs"

# Extraindo os PDFs
os.makedirs(extracted_path, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

# Função para extrair texto de PDFs
def extract_text_from_pdfs(pdf_folder):
    text_data = ""
    for pdf_file in os.listdir(pdf_folder):
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, pdf_file)
            doc = pymupdf.open(pdf_path)
            for page in doc:
                text_data += page.get_text("text") + "\n\n"
    return text_data

# Extraindo texto dos PDFs
medical_texts = extract_text_from_pdfs(extracted_path)

# Salvando em um arquivo de texto
medical_texts_path = "medical_texts.txt"
with open(medical_texts_path, "w", encoding="utf-8") as f:
    f.write(medical_texts)

# Retornar o caminho do arquivo salvo
medical_texts_path


# Criando objeto do RAG

In [4]:
from langchain_community.chat_models import ChatOllama
from langchain.schema import SystemMessage, HumanMessage
from langchain_community.vectorstores.faiss import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langgraph.graph import StateGraph, START, END
from pydantic import BaseModel, Field
import numpy as np


# Carregar embeddings do Sentence-Transformers
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Carregar e processar documentos médicos (já extraídos dos PDFs)
loader = TextLoader("medical_texts.txt")  # Arquivo consolidado com informações médicas extraídas
texts = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(texts)

# Criar banco vetorial FAISS
vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()

# Configurar modelo LLM
llm = ChatOllama(model="llava:7b")  # Trocar conforme necessário

<ipython-input-4-e838ba316853>:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  war

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-4-e838ba316853>:27: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llava:7b")  # Trocar conforme necessário


# Leitura do modelo selecionado

In [5]:
from Image_Segmentation.network import U_Net,R2U_Net,AttU_Net,R2AttU_Net
import torch
import torch.nn.functional as F
from torchvision import transforms as T
import random
import base64
from PIL import Image
from io import BytesIO


model = U_Net(img_ch=3,output_ch=1)
model.load_state_dict(torch.load('./U_Net-100-0.0005-70-0.6667.pkl', map_location=torch.device('cpu')))

<ipython-input-5-1d7242dc4270>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./U_Net-100-0.0005-70-0.6667.pkl', map_location=torch.devi

<All keys matched successfully>

# Criação dos agentes e fluxo

In [6]:
# Configurar agentes
class SegmentationAgent:
  def __init__(self,model,threshold=.5,image_size=224,mode='train',augmentation_prob=0.4):
    self.model = model
    self.image_size = image_size
    self.mode = mode
    self.RotationDegree = [0,90,180,270]
    self.augmentation_prob = augmentation_prob

  def convert_to_base64(self, pil_image):
    """
    Convert PIL images to Base64 encoded strings

    :param pil_image: PIL image
    :return: Base64 string
    """
    pil_image = pil_image.convert("RGB")
    buffered = BytesIO()
    pil_image.save(buffered, format="JPEG")  # You can change the format if needed
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

  def segment_image(self, state):
    """Reads an image from a file and preprocesses it and returns."""
    img = Image.open(state.image_path)
    encoded_string = self.convert_to_base64(img)

    aspect_ratio = img.size[1]/img.size[0]

    Transform = []

    ResizeRange = random.randint(300,320)
    Transform.append(T.Resize((int(ResizeRange*aspect_ratio),ResizeRange)))
    p_transform = random.random()

    Transform.append(T.Resize((int(256*aspect_ratio)-int(256*aspect_ratio)%16,256)))
    Transform.append(T.ToTensor())
    Transform = T.Compose(Transform)
    img_ = img
    img = Transform(img)

    # Norm_ = T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    # img = Norm_(img)

    SR = model(img.unsqueeze(0))
    SR_s = F.sigmoid(SR)

    SR_image = (SR_s>.4)*img
    # im = Image.fromarray(SR_image[0][0].detach().numpy())
    im = T.ToPILImage()(SR_image[0])
    seg_encoded_string = self.convert_to_base64(im)

    return {"seg_image_data": seg_encoded_string, "image_data": encoded_string}
#tensor_image = SegmentationAgent('/content/image/ISIC_0036209.jpg', model).segment_image()
#plt.imshow(tensor_image[0].permute(1, 2, 0))

class DiagnosticAgent:
    def __init__(self, llm):
        self.llm = llm

    def analyze_lesion(self, state):
      content_str = """
      Duas imagens foram fornecidas para análise:

      1️⃣ Imagem original da lesão de pele, que mantém seu contexto visual completo.
      2️⃣ Imagem segmentada, que realça os contornos e características internas da lesão.

      Descreva as características visíveis com foco em textura, coloração, simetria e bordas. Se houver padrões
      reconhecíveis, detalhe-os comparando com formas naturais já conhecidas.

      Dê prioridade às estruturas internas destacadas na segunda imagem, mas use a primeira para contexto geral
      """
      content = [
          # {'type': 'image_url', 'image_url': f'data:image/jpeg;base64,{state.image_data}'},
          {'type': 'text', 'text': "Aqui está a imagem original da lesão:"},
          {'type': 'image_url', 'image_url': f'data:image/jpeg;base64,{state.image_data}'},
          {'type': 'text', 'text': "Aqui está a versão segmentada da lesão:Aqui está a versão segmentada da lesão:"},
          {'type': 'image_url', 'image_url': f'data:image/jpeg;base64,{state.seg_image_data}'},
          {'type': 'text', 'text': content_str}
      ]
      msg = [HumanMessage(content=content)]
      return {"diagnosis": llm.invoke(msg).content}

class PrognosticAgent:
    def __init__(self, llm):
        self.llm = llm

    def assess_risk(self, state):
      content_str="""
      Duas imagens foram fornecidas para análise:
      1️⃣ Imagem original da lesão de pele, preservando sua aparência natural.
      2️⃣ Imagem segmentada, que destaca áreas de interesse na lesão.
      Com base em ambas as imagens, descreva possíveis padrões evolutivos da lesão ao longo do tempo. Considere fatores
      como variações na coloração, bordas, simetria e possíveis mudanças estruturais. Compare essas características com
      padrões documentados na literatura médica sobre a progressão de condições dermatológicas. Relate quais indícios
      visuais podem sugerir maior ou menor risco de agravamento.
      """
      content = [
          {'type': 'text', 'text': "Aqui está a imagem original da lesão:"},
          {'type': 'image_url', 'image_url': f'data:image/jpeg;base64,{state.image_data}'},
          {'type': 'text', 'text': "Aqui está a versão segmentada da lesão:Aqui está a versão segmentada da lesão:"},
          {'type': 'image_url', 'image_url': f'data:image/jpeg;base64,{state.seg_image_data}'},
          {'type': 'text', 'text': content_str}
      ]
      msg = [HumanMessage(content=content)]
      return {"prognosis": llm.invoke(msg).content}

class SummaryAgent:
    def __init__(self, llm):
        self.llm = llm

    def summarize(self, state):
      content_str=f"""Com base nas descrições fornecidas pelos especialistas, compile um relatório estruturado sobre a lesão analisada. O relatório deve incluir:
1️⃣ Características Visuais: Resumo das observações sobre textura, coloração, bordas e padrões internos da lesão.
2️⃣ Análise Comparativa: Como essas características se relacionam com padrões conhecidos de lesões dermatológicas.
3️⃣ Estimativa de Evolução: Possíveis mudanças na lesão ao longo do tempo, considerando padrões documentados.
4️⃣ Recomendações: Medidas sugeridas para acompanhamento, investigações adicionais ou intervenção clínica, se aplicável.

Apresente o relatório em formato JSON, garantindo clareza e organização das informações.
      [[{state.diagnosis}]]
      <<{state.prognosis}>>
      """
      content = [
          {'type': 'text', 'text': "Aqui está a imagem original da lesão:"},
          {'type': 'image_url', 'image_url': f'data:image/jpeg;base64,{state.image_data}'},
          {'type': 'text', 'text': "Aqui está a versão segmentada da lesão:Aqui está a versão segmentada da lesão:"},
          {'type': 'image_url', 'image_url': f'data:image/jpeg;base64,{state.seg_image_data}'},
          {'type': 'text', 'text': content_str}
      ]
      msg = [HumanMessage(content=content)]
      return {"validation": llm.invoke(msg).content}

# Configurar agente crítico
class CriticalReviewAgent:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm

    def validate_diagnosis(self, state):
        retrieved_docs = self.retriever.invoke(state.diagnosis)
        return {"final_report": f"Confirmação baseada em literatura médica: {retrieved_docs[0].page_content[:200]}..."}


# Definir o esquema de estado inicial
class GraphState(BaseModel):
    image_path: str
    image_data: str = None
    seg_image_data: str = None
    diagnosis: str = None
    prognosis: str = None
    validation: str = None
    final_report: str = None

# Criar fluxo no LangGraph
graph = StateGraph(GraphState)

graph.add_node("segmentation", SegmentationAgent(model).segment_image)
graph.add_node("diagnostic", DiagnosticAgent(llm).analyze_lesion)
graph.add_node("prognostic", PrognosticAgent(llm).assess_risk)
graph.add_node("summary", SummaryAgent(llm).summarize)
graph.add_node("critical_review", CriticalReviewAgent(retriever, llm).validate_diagnosis)

# Definir conexões do fluxo
graph.add_edge(START, "segmentation")
graph.add_edge("segmentation", "diagnostic")
graph.add_edge("diagnostic", "prognostic")
graph.add_edge("prognostic", "summary")
graph.add_edge("summary", "critical_review")
graph.add_edge("critical_review", END)

g = graph.compile()

# Execução do agente

In [9]:
import os
import json


list_image = ['/content/Image/'+path for path in os.listdir('/content/Image') if path[-3:]=='jpg']
result_dict = dict()
for path in list_image:
  for event in g.stream({"image_path": path}, stream_mode="values"):
    for e in ['image_data', 'seg_image_data']:
      event.pop(e, None)
    result_dict[path] = event
    with open("llm_result.json", "w", encoding="utf-8") as arquivo:
      json.dump(result_dict, arquivo, indent=4, ensure_ascii=False)

In [78]:
for event in g.stream({"image_path": '/content/Image/ISIC_0012572.jpg'}, stream_mode="values"):
  pass

In [79]:
print("Diagnóstico:\n\t", event['diagnosis'],'\n', 100*"=",'\n')
print("Prognóstico:\n\t", event['prognosis'],'\n', 100*"=",'\n')
print("Relatório:\n\t", event['validation'])

Diagnóstico:
	  Na imagem, vemos uma lesão com características que podem ser indicativas do câncer de pele, incluindo:

1. **Limitação**: A lesão está delimitada por bordas nítidas e contínuas, possivelmente ocorro da borda natural da pele.

2. **Coloração**: O tecido envolvente à lesão parece ser um tecido que não é típico de uma pele normal. Há tonalidades mais claras e clara ao redor da borda, sugerindo uma área circundante intacta ou possivelmente lesionada, mas ainda com aparência do tecido normal.

3. **Textura**: A textura da lesão é irregular e irregularmente elevada em alguns locais, possivelmente indicando um crescimento rápido de células cancerosas.

4. **Borda irregular**: A borda da lesão não está bem definida, sugerindo uma lesão com um crescimento incontrolado e desordenado.

5. **Centro sanguíneo**: Há o que parece ser um centro sangrante no meio da lesão, o que é comum em alguns tipos de tumores como o carcinoide. Este tipo de lesão pode ser uma forma de verificar se h